In [1]:
#!pip install tensorflow==1.11.0
import pandas as pd
import numpy as np
import os 
import datetime
import pickle
from copy import deepcopy
import collections
import re
import random
from pathlib import Path
import spacy
import nltk
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')

pd.set_option('display.max_rows', 5)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amogh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
posts = pd.read_csv('typed_posts.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,27,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
posts.columns

Index(['created_utc', 'subreddit', 'author', 'domain', 'url', 'num_comments',
       'score', 'ups', 'downs', 'title', 'selftext', 'saved', 'id',
       'from_kind', 'gilded', 'from', 'stickied', 'retrieved_on', 'over_18',
       'thumbnail', 'subreddit_id', 'hide_score', 'link_flair_css_class',
       'author_flair_css_class', 'archived', 'is_self', 'from_id', 'permalink',
       'name', 'author_flair_text', 'quarantine', 'link_flair_text',
       'distinguished', 'type'],
      dtype='object')

In [0]:
posts.type.unique()

array(['intj', 'intp', 'enfp', 'infp', 'infj', 'istj', 'istp', 'enfj',
       'entp', 'entj', 'estp', 'isfj', 'isfp', 'estj', nan, 'esfp',
       'esfj'], dtype=object)

In [0]:
len(posts.author.unique())

10021

In [0]:
posts.shape

(387659, 34)

In [0]:
len(posts.subreddit.unique())

20310

In [0]:
mbti_data = pd.read_csv('mbti9k_comments.csv', nrows=1000)
mbti_data = mbti_data.drop_duplicates(subset=['author'], keep='first') #103 authors are duplicated with different types
mbti_data = mbti_data[['author', 'comment', 'type']]

In [0]:
mbti_data[0:2]

,author,comment,type
0,--asmodeus--,Yes! Personally I feel that as helpful as it i...,entp
1,--hypnos--,I guess I'd stick to being a lesbian so I woul...,intj


In [0]:
len(mbti_data.author.unique())

9149

In [0]:
mbti_data.author.unique()[0:5]

array(['--asmodeus--', '--hypnos--', '-Afrodisiac-', '-Automaticity',
       '-Avacyn'], dtype=object)

In [0]:
mbti_data.shape

(9149, 7)

In [0]:
def docSplit(row, n=100):

  text = row['comment']
  text = textClean(text)
  text = nltk.word_tokenize(text)
  text_ =  [' '.join(text[i:i+n]) for i in range(0, len(text), n)]
  text_ = "|".join(text_)
  
  return text_

def removeSmallSent(row):
  text = row['comment']
  text = text.split(' ')
  if len(text) < 50:
    return -1
  else: 
    text = ' '.join(text).strip(' ')
    text = re.sub(r'\s([?.!"](?:\s|$))', r'\1', text)
    return text

def textClean(s):
  s = re.sub(r'\s*(?:https?://)?www\.\S*\.[A-Za-z]{2,5}\s*', ' ', s).strip()
  return s

mbti_data['comment'] = mbti_data.apply(lambda x: docSplit(x), axis=1)

mbti_data = (mbti_data.set_index(['author', 'type'])
   .apply(lambda x: x.str.split('|').explode())
   .reset_index()) 
mbti_data['comment'] = mbti_data.apply(lambda x: removeSmallSent(x), axis=1)
mbti_data = mbti_data[mbti_data['comment'] != -1]

In [6]:
len(mbti_data)

26648

In [7]:
mbti_data['comment'].tolist()[0:10]

["Yes! Personally I feel that as helpful as it is to meticulously plan out purchases , it often sucks the joy out of getting a new product and creates even more of an obsessive edge to things. Hope I can get back to the mindset of buying something fun without stressing over the implications of it Ooooh thanks for the suggestion! I 'm drawn to Tigerlily even though it might not work for me haha Lol definitely! I 'm great at convincing other people to buy makeup even though when it comes to myself I can never bring myself to pay the money Thank you! ! I loved both of these at Sephora today , but I did n't have time to play with colors Ooooh thanks for the suggestion! I 'm drawn to Tigerlily even though it might not work for me haha",
 "I think it 's because lip colors tend to be opaque and getting the exact shade right is crucial because anything slightly weird will just throw your whole look off. Blushes and eyeshadows tend to be sheer and therefore are affected much more by your underl

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

--2020-04-18 04:31:50--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.128, 2607:f8b0:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  61.3MB/s    in 6.3s    

2020-04-18 04:31:56 (62.1 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

--2020-04-18 04:31:57--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [te

In [38]:
%tensorflow_version 1.11
import tensorflow as tf
import modeling
import optimization
import run_classifier
import tokenization

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.11`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
import zipfile
folder = 'model_folder'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [40]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')

>> Model output directory: model_folder/outputs
>>  BERT pretrained directory: model_folder/uncased_L-12_H-768_A-12


In [0]:
df = pd.DataFrame()
df["Text"] = mbti_data['comment']
df["Label"] = LabelEncoder().fit_transform(mbti_data['type'])

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["Text"].values, df["Label"].values, test_size=0.2, random_state=42)

In [0]:
set(y_test)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [0]:
def create_examples(lines, set_type, labels=None):
  #Generate data for the BERT model
  guid = f'{set_type}'
  examples = []
  if guid == 'train':
      for line, label in zip(lines, labels):
          text_a = line
          label = str(label)
          examples.append(
            run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
  else:
      for line in lines:
          text_a = line
          label = '0'
          examples.append(
            run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
  return examples

In [26]:
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 100
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(i) for i in sorted(df['Label'].unique())]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'model_folder/outputs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f34d403d0b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100000, num_shards=8, num_cores_per_repli

In [0]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

Streaming output truncated to the last 5000 lines.
INFO:tensorflow:global_step/sec: 0.0899204
INFO:tensorflow:examples/sec: 1.43873
INFO:tensorflow:global_step/sec: 0.0891551
INFO:tensorflow:examples/sec: 1.42648
INFO:tensorflow:global_step/sec: 0.0900363
INFO:tensorflow:examples/sec: 1.44058
INFO:tensorflow:global_step/sec: 0.0898082
INFO:tensorflow:examples/sec: 1.43693
INFO:tensorflow:global_step/sec: 0.0903044
INFO:tensorflow:examples/sec: 1.44487
INFO:tensorflow:global_step/sec: 0.0903466
INFO:tensorflow:examples/sec: 1.44555
INFO:tensorflow:global_step/sec: 0.0900301
INFO:tensorflow:examples/sec: 1.44048
INFO:tensorflow:global_step/sec: 0.089817
INFO:tensorflow:examples/sec: 1.43707
INFO:tensorflow:global_step/sec: 0.0898662
INFO:tensorflow:examples/sec: 1.43786
INFO:tensorflow:global_step/sec: 0.0894273
INFO:tensorflow:examples/sec: 1.43084
INFO:tensorflow:global_step/sec: 0.0898892
INFO:tensorflow:examples/sec: 1.43823
INFO:tensorflow:global_step/sec: 0.0908326
INFO:tensorflow:

In [0]:
X_train[0:10]

array(["is not quite true they Downvote , that is Reddit.Likewise.128 tick is better for low latency environments while playing with over 30ms on a bad optimized Server will help you nothing.In fact most of the Hitreg problems are due to the poor optimization of the 64tick server instances on a Server.You might want to read this : https : //developer.valvesoftware.com/wiki/Source_Multiplayer_Networking ? language=enhttp : //mukunda.com/128tick.htmlI hope this will help you understand why these wishes are just wishes and will stay wishes . 244 DMG/LE , MM rank means shit though ; D Nope because nobody runs with a knife against an ak while a noscope is something you can do 'safer ' . Uhm , you know you ca n't apply that to accounts right ? Server login tokens are basically treated as accounts . A token is a single Account compared to normal users . And normal users get banned",
       "be right there . That would be the least visible and fastest window for me to draw in.And definitely , 

In [0]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 500

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn



In [28]:
predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

INFO:tensorflow:Writing example 0 of 5330
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test
INFO:tensorflow:tokens: [CLS] a well documented effect of e . assuming e in the 90s is the same as e now . that is not a uniquely american trait . russia is n ' t invading ukraine because putin feels bad for the russian minority , and cyrus the great did [SEP]
INFO:tensorflow:input_ids: 101 1037 2092 8832 3466 1997 1041 1012 10262 1041 1999 1996 17233 2003 1996 2168 2004 1041 2085 1012 2008 2003 2025 1037 20640 2137 18275 1012 3607 2003 1050 1005 1056 17657 5924 2138 22072 5683 2919 2005 1996 2845 7162 1010 1998 16123 1996 2307 2106 102
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test
INFO:t

In [29]:
preds = []
for prediction in result:
  preds.append(np.argmax(prediction['probabilities']))

{}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 50)
INFO:tensorflow:  name = input_mask, shape = (?, 50)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 50)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/l

In [30]:
preds[0:10]

[3, 2, 3, 2, 2, 3, 2, 2, 6, 2]

In [31]:
from sklearn.metrics import accuracy_score
print("Accuracy of BERT is:",accuracy_score(y_test,preds))


Accuracy of BERT is: 0.04127579737335835


In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test,preds))



              precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.00      0.00      0.00        59
           2       0.02      0.24      0.04       192
           3       0.03      0.38      0.06       182
           4       0.00      0.00      0.00       251
           5       0.11      0.02      0.04       788
           6       0.19      0.06      0.09      1387
           7       0.00      0.00      0.00      1990
           8       0.00      0.00      0.00        10
           9       0.03      0.03      0.03       272
          10       0.00      0.00      0.00       164

    accuracy                           0.04      5330
   macro avg       0.03      0.07      0.02      5330
weighted avg       0.07      0.04      0.03      5330



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
